In [1]:
# 基本パッケージ（numpy,Pandas,matplotlib）
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# globのimport
import glob

In [2]:
# 表示関連
# DataFrameの列数設定
pd.set_option('display.max_columns', 500)

In [3]:
# 2001～2018年の株価データをマージする
## globでファイル名の一覧を取得
stock_price_files = glob.glob('stockPrice/*.csv')
stock_price_list = []

# ファイルを読み込み、DataFrameでlistに格納する。
for f in stock_price_files:
    stock_price_list.append(pd.read_csv(f, header=1, encoding="shift-jis", parse_dates = [0]))
    
# Listに格納されたデータを全てconcat関数で連結    
stock_price_all = pd.concat(stock_price_list)

# 列名の変更
stock_price_all.rename(columns = {'日付':'Date','始値':'Open','高値':'High','安値':'Low','終値':'Close','終値調整値':'Adj Close',
                                  '出来高':'Volume'}, inplace = True)
# 移動平均（5日）の作成
#stock_price_all['ma_5'] = stock_price_all['Adj Close'].rolling(5).mean()

# EMAの作成
stock_price_all['eam_5'] = stock_price_all['Adj Close'].ewm(span=5).mean()

# MACDの作成
stock_price_all['macd'] = stock_price_all['Adj Close'].ewm(span=5).mean() - stock_price_all['Adj Close'].ewm(span=25).mean()

# ボリンジャーバンドの作成
#stock_price_all['bband_upper'] = stock_price_all['ma'] +(stock_price_all['Adj Close'].rolling(5).std()*2)
#stock_price_all['bband_lower'] = stock_price_all['ma'] -(stock_price_all['Adj Close'].rolling(5).std()*2)

# 移動平均乖離率の作成
creating_feature = pd.DataFrame({'Date' : stock_price_all['Date'] })
#creating_feature['pvma_5'] = (stock_price_all['Adj Close'] - stock_price_all['ma_5'])/stock_price_all['ma_5']
creating_feature['pveam_5'] = (stock_price_all['Adj Close'] - stock_price_all['eam_5'])/stock_price_all['eam_5']

# モメンタム（10日）の作成
#creating_feature['momentum'] = stock_price_all['Adj Close'].diff(10).apply( lambda x: 0 if x < 0 else 1 )

# 移動平均と終値の関係
#creating_feature['cross_5'] =(stock_price_all['Adj Close'] - stock_price_all['ma_5']).apply( lambda x: 0 if x < 0 else 1 )
#creating_feature['cross_25'] =(stock_price_all['Adj Close'] - stock_price_all['ma_25']).apply( lambda x: 0 if x < 0 else 1 )
#creating_feature['cross_ma'] =(stock_price_all['ma_5'] - stock_price_all['ma_25']).apply( lambda x: 0 if x < 0 else 1 )


# 欠損値のチェック
print(stock_price_all.isnull().sum())

# 読み込みデータの確認
stock_price_all.head()

Date         0
Open         0
High         0
Low          0
Close        0
Volume       0
Adj Close    0
eam_5        0
macd         0
dtype: int64


,Date,Open,High,Low,Close,Volume,Adj Close,eam_5,macd
0,2001-07-13,12490,12490,12300,12310,259430,12310,12310.000000,0.000000
1,2001-07-16,12400,12400,12250,12330,99740,12330,12322.000000,1.600000
2,2001-07-17,12170,12170,12100,12120,93150,12120,12226.315789,-21.871844
3,2001-07-18,12150,12150,11830,11840,165330,11840,12065.846154,-67.735955
4,2001-07-19,11990,11990,11860,11920,620220,11920,12009.857820,-73.911543


In [4]:
# creating_featureの確認
creating_feature.head()

,Date,pveam_5
0,2001-07-13,0.000000
1,2001-07-16,0.000649
2,2001-07-17,-0.008696
3,2001-07-18,-0.018718
4,2001-07-19,-0.007482


In [5]:
# 2002～2018年の為替レートを取得
market_Data = pd.read_csv('market/market.csv', header=0, encoding="shift-jis", parse_dates = [0])

# 不要な通貨を削除
#market_Data.drop(columns=['USD','SEK','DKK','NOK','AUD','CNY','HKD','PHP','SGD','KWD','SAR','AED','TWD'], inplace=True)

# 欠損値のチェック
print(market_Data.isnull().sum())

# レート情報をずらす
market_Data = market_Data.shift(-1)

# 読み込みデータの表示
market_Data.head()

Date        0
USD         0
GBP         0
EUR         0
CAD         0
CHF         0
SEK         0
DKK         0
NOK         0
AUD         0
NZD         0
ZAR         0
BHD         0
IDR(100)    0
CNY         0
HKD         0
INR         0
PHP         0
SGD         0
KRW(100)    0
THB         0
KWD         0
SAR         0
AED         0
MXN         0
TWD         0
dtype: int64


,Date,USD,GBP,EUR,CAD,CHF,SEK,DKK,NOK,AUD,NZD,ZAR,BHD,IDR(100),CNY,HKD,INR,PHP,SGD,KRW(100),THB,KWD,SAR,AED,MXN,TWD
0,2002-04-02,133.20,191.78,117.18,83.38,80.15,13.00,15.77,15.24,71.02,58.86,11.78,353.78,1.38,16.09,17.08,2.73,2.62,72.18,10.12,3.06,435.01,35.53,36.27,14.80,3.82
1,2002-04-03,133.20,191.26,116.96,83.65,80.02,12.95,15.75,15.23,71.14,58.81,11.89,353.78,1.37,16.09,17.08,2.74,2.62,72.12,10.03,3.04,436.58,35.53,36.27,14.77,3.82
2,2002-04-04,133.10,191.13,117.15,83.72,80.18,12.93,15.77,15.33,70.81,58.52,12.02,353.52,1.37,16.08,17.06,2.73,2.61,72.26,10.02,3.05,435.11,35.50,36.24,14.75,3.82
3,2002-04-05,132.30,189.74,116.32,82.96,79.47,12.87,15.65,15.22,70.33,58.36,11.83,351.39,1.37,15.98,16.96,2.71,2.60,71.85,10.03,3.04,432.21,35.29,36.03,14.66,3.80
4,2002-04-08,131.55,188.54,115.69,82.80,79.00,12.82,15.57,15.13,69.76,57.43,11.92,349.40,1.38,15.89,16.87,2.70,2.58,71.76,9.90,3.03,429.62,35.09,35.81,14.62,3.77


In [6]:
# 2002～2018年の主要指数データをマージする
## globでファイル名の一覧を取得
world_indices_files = glob.glob('worldIndices/*.csv')

# ファイルを読み込み、DataFrameでlistに格納する。
for i in range(len(world_indices_files)):
    if i == 0:
         world_indices_data = pd.read_csv(world_indices_files[i], header=0, encoding='shift-jis', parse_dates = [0])
         world_indices_data.drop(columns=['Close','Volume'], inplace=True)
         # 欠損値の補完
         world_indices_data = world_indices_data.fillna(method='ffill')
         # EMAの作成
         world_indices_data['eam_5'] = world_indices_data['Adj Close'].ewm(span=5).mean()
         # MACDの作成
         world_indices_data['macd'] = world_indices_data['Adj Close'].ewm(span=5).mean() -world_indices_data['Adj Close'].ewm(span=25).mean()
         # 移動平均乖離率の作成
         creating_feature['pveam_5'] = (world_indices_data['Adj Close'] - world_indices_data['eam_5'])/world_indices_data['eam_5']
         #カラム名の変更
         world_indices_data.rename(columns = {'Open':'Open'+ world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'High':'High'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'Low':'Low'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'Adj Close':'Adj Close'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'eam_5':'eam_5'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'macd':'macd'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'Volume':'Volume'+world_indices_files[i].replace('worldIndices\\','').replace('.csv','')}, inplace = True)
         creating_feature.rename(columns = {'pveam_5':'pveam_5'+world_indices_files[i].replace('worldIndices\\','').replace('.csv','')}, inplace = True)
    else:      
        work_data = pd.read_csv(world_indices_files[i], header=0, encoding='shift-jis', parse_dates = [0])
        work_data.drop(columns=['Close','Volume'], inplace=True)
        # 欠損値の補完
        work_data = work_data.fillna(method='ffill')
        # EMAの作成
        work_data['eam_5'] = work_data['Adj Close'].ewm(span=5).mean()
        # MACDの作成
        work_data['macd'] = work_data['Adj Close'].ewm(span=5).mean() - work_data['Adj Close'].ewm(span=25).mean()
        # 移動平均乖離率の作成
        creating_feature['pveam_5'] = (work_data['Adj Close'] - work_data['eam_5'])/work_data['eam_5']
        #カラム名の変更
        work_data.rename(columns = {'Open':'Open'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'High':'High'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'Low':'Low'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'Adj Close':'Adj Close'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'eam_5':'eam_5'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'macd':'macd'+world_indices_files[i].replace('worldIndices\\','').replace('.csv',''),
                                    'Volume':'Volume'+world_indices_files[i].replace('worldIndices\\','').replace('.csv','')}, inplace = True)
        creating_feature.rename(columns = {'pveam_5':'pveam_5'+world_indices_files[i].replace('worldIndices\\','').replace('.csv','')}, inplace = True)
        world_indices_data = pd.merge(world_indices_data, work_data, on='Date', how='left')

# 欠損値のチェック
print(world_indices_data.isnull().sum())

# 読み込みデータの表示
world_indices_data.head()

Date                    0
Open000001.SS           0
High000001.SS           0
Low000001.SS            0
Adj Close000001.SS      0
eam_5000001.SS          0
macd000001.SS           0
Open^AORD             449
High^AORD             449
Low^AORD              449
Adj Close^AORD        449
eam_5^AORD            449
macd^AORD             449
Open^AXJO             449
High^AXJO             449
Low^AXJO              449
Adj Close^AXJO        449
eam_5^AXJO            449
macd^AXJO             449
Open^BFX               46
High^BFX               46
Low^BFX                46
Adj Close^BFX          46
eam_5^BFX              46
macd^BFX               46
Open^BSESN            161
High^BSESN            161
Low^BSESN             161
Adj Close^BSESN       161
eam_5^BSESN           161
                     ... 
Open^NYA              144
High^NYA              144
Low^NYA               144
Adj Close^NYA         144
eam_5^NYA             144
macd^NYA              144
Open^RUT              134
High^RUT    

,Date,Open000001.SS,High000001.SS,Low000001.SS,Adj Close000001.SS,eam_5000001.SS,macd000001.SS,Open^AORD,High^AORD,Low^AORD,Adj Close^AORD,eam_5^AORD,macd^AORD,Open^AXJO,High^AXJO,Low^AXJO,Adj Close^AXJO,eam_5^AXJO,macd^AXJO,Open^BFX,High^BFX,Low^BFX,Adj Close^BFX,eam_5^BFX,macd^BFX,Open^BSESN,High^BSESN,Low^BSESN,Adj Close^BSESN,eam_5^BSESN,macd^BSESN,Open^BVSP,High^BVSP,Low^BVSP,Adj Close^BVSP,eam_5^BVSP,macd^BVSP,Open^DJI,High^DJI,Low^DJI,Adj Close^DJI,eam_5^DJI,macd^DJI,Open^FCHI,High^FCHI,Low^FCHI,Adj Close^FCHI,eam_5^FCHI,macd^FCHI,Open^GDAXI,High^GDAXI,Low^GDAXI,Adj Close^GDAXI,eam_5^GDAXI,macd^GDAXI,Open^GSPC,High^GSPC,Low^GSPC,Adj Close^GSPC,eam_5^GSPC,macd^GSPC,Open^GSPTSE,High^GSPTSE,Low^GSPTSE,Adj Close^GSPTSE,eam_5^GSPTSE,macd^GSPTSE,Open^HSI,High^HSI,Low^HSI,Adj Close^HSI,eam_5^HSI,macd^HSI,Open^IPSA,High^IPSA,Low^IPSA,Adj Close^IPSA,eam_5^IPSA,macd^IPSA,Open^IXIC,High^IXIC,Low^IXIC,Adj Close^IXIC,eam_5^IXIC,macd^IXIC,Open^JKSE,High^JKSE,Low^JKSE,Adj Close^JKSE,eam_5^JKSE,macd^JKSE,Open^KS11,High^KS11,Low^KS11,Adj Close^KS11,eam_5^KS11,macd^KS11,Open^MERV,High^MERV,Low^MERV,Adj Close^MERV,eam_5^MERV,macd^MERV,Open^MXX,High^MXX,Low^MXX,Adj Close^MXX,eam_5^MXX,macd^MXX,Open^N100,High^N100,Low^N100,Adj Close^N100,eam_5^N100,macd^N100,Open^NYA,High^NYA,Low^NYA,Adj Close^NYA,eam_5^NYA,macd^NYA,Open^RUT,High^RUT,Low^RUT,Adj Close^RUT,eam_5^RUT,macd^RUT,Open^TWII,High^TWII,Low^TWII,Adj Close^TWII,eam_5^TWII,macd^TWII,Open^VIX,High^VIX,Low^VIX,Adj Close^VIX,eam_5^VIX,macd^VIX,Open^XAX,High^XAX,Low^XAX,Adj Close^XAX,eam_5^XAX,macd^XAX
0,2002-04-01,1598.026001,1609.916992,1589.406982,1608.505981,1608.505981,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3482.939941,3519.320068,3482.939941,3500.179932,3500.179932,0.000000,13256.0,13499.0,13068.0,13467.0,13467.000000,0.000000,10402.070313,10402.070313,10263.679688,10362.700195,10362.700195,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1147.390015,1147.839966,1132.869995,1146.540039,1145.787094,2.278342,7854.000000,7932.100098,7850.000000,7888.500000,7888.500000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1163.630005,1164.579956,1149.910034,1151.329956,1151.329956,0.000000,1834.589966,1865.369995,1817.250000,1862.619995,1862.619995,0.000000,482.524994,486.667999,481.006989,486.653564,486.653564,0.000000,900.469971,900.469971,874.900024,875.830017,875.830017,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,7362.140137,7427.160156,7341.350098,7371.890137,7371.890137,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,6327.109863,6327.109863,6327.109863,6327.109863,6327.109863,0.000000,506.459991,506.459991,498.399994,504.500000,504.500000,0.000000,6118.450195,6213.319824,6118.450195,6186.416504,6186.416504,0.000000,18.990000,19.500000,18.610001,18.730000,18.730000,0.000000,910.489990,915.210022,910.270020,914.750000,914.750000,0.000000
1,2002-04-02,1608.519043,1610.727051,1575.972046,1582.883057,1593.132227,-2.049834,3363.300049,3363.300049,3332.399902,3338.500000,3338.50000,0.000000,3414.800049,3418.000000,3380.199951,3387.300049,3387.300049,0.000000,2766.360107,2766.360107,2740.689941,2755.050049,2755.050049,0.000000,3505.399902,3537.290039,3490.870117,3505.790039,3503.545996,0.448809,13482.0,13535.0,13245.0,13245.0,13333.800000,-17.760000,10352.459961,10352.459961,10264.860352,10313.709961,10333.306055,-3.919219,4673.029785,4688.160156,4626.459961,4627.330078,4627.330078,0.000000,5379.640137,5379.640137,5289.799805,5311.080078,5311.080078,0.000000,1146.540039,1146.540039,1135.709961,1136.760010,1142.778066,-0.207588,7895.899902,7920.100098,7886.700195,7893.399902,7891.439941,0.391992,11032.360352,11055.980469,10816.730469,10878.040039,10878.040039,0.000000,1150.619995,1151.089966,1139.849976,1139.969971,1144.513965,-0.908799,1836.030029,1839.369995,1804.400024,1804.400024,1827.688012,-4.657598,487.178009,491.790009,485.949005,489.074493,488.106121,0.193674,878.900024,905.340027,871.450012,905.340027,893.536023,2.360801,436.220001,447.399994,432.750000,435.089996,435.089996,0.

In [7]:
# 内部結合で結合する
merge_work= pd.merge(world_indices_data,market_Data, on='Date', how='inner')
merge_Data = pd.merge(stock_price_all, merge_work, on='Date', how='inner')

In [8]:
# 目的変数の作成
merge_Data['diff'] = merge_Data['Close'] - merge_Data['Open']
answersData = pd.DataFrame({'Date' : merge_Data['Date']})
answersData['answer'] =merge_Data['diff'].apply( lambda x: 0 if x < 0 else 1 )

# 不要な列を削除
merge_Data.drop(columns=['Date','Close','diff'], inplace=True)

# 各列を変化率へ一括変換
merge_Data_pct = merge_Data.pct_change()

# 作製した特徴量と目的変数の付与
make_data = pd.merge(creating_feature, answersData, on='Date', how='inner')
make_data.drop(columns=['Date'], inplace=True)
stock_Price_Prediction = pd.concat([merge_Data_pct, make_data], axis=1)

# 正解ラベルをずらし、説明変数から見て未来の値とする
stock_Price_Prediction.answer = stock_Price_Prediction.answer.shift(-1)


In [9]:
# 欠損値のチェック
print(stock_Price_Prediction.isnull().sum())

# NaN,INF,-INFは削除する
stock_Price_Prediction = stock_Price_Prediction.replace([np.inf, -np.inf], np.nan)
stock_Price_Prediction = stock_Price_Prediction.dropna()

# CSVへ変換
stock_Price_Prediction.to_csv("stock_Price_Prediction_v3.11.csv",index = False, encoding="shift-jis")

Open                  1
High                  1
Low                   1
Volume                1
Adj Close             1
eam_5                 1
macd                  1
Open000001.SS         1
High000001.SS         1
Low000001.SS          1
Adj Close000001.SS    1
eam_5000001.SS        1
macd000001.SS         1
Open^AORD             1
High^AORD             1
Low^AORD              1
Adj Close^AORD        1
eam_5^AORD            1
macd^AORD             1
Open^AXJO             1
High^AXJO             1
Low^AXJO              1
Adj Close^AXJO        1
eam_5^AXJO            1
macd^AXJO             1
Open^BFX              1
High^BFX              1
Low^BFX               1
Adj Close^BFX         1
eam_5^BFX             1
                     ..
KWD                   1
SAR                   1
AED                   1
MXN                   1
TWD                   1
pveam_5000001.SS      0
pveam_5^AORD          0
pveam_5^AXJO          0
pveam_5^BFX           0
pveam_5^BSESN         0
pveam_5^BVSP    